# Introduction to Natural Language Processing 2 Lab02

In [1]:
%%capture
%matplotlib inline
!pip install torch==1.13.0 spacy sacrebleu torchdata torchtext -U
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm


# Language Translation with nn.Transformer and torchtext

This tutorial shows:
    - How to train a translation model from scratch using Transformer. 
    - Use tochtext library to access  [Multi30k](http://www.statmt.org/wmt16/multimodal-task.html#task1)_ dataset to train a German to English translation model.


## Data Sourcing and Processing

[torchtext library](https://pytorch.org/text/stable/)_ has utilities for creating datasets that can be easily
iterated through for the purposes of creating a language translation
model. In this example, we show how to use torchtext's inbuilt datasets, 
tokenize a raw text sentence, build vocabulary, and numericalize tokens into tensor. We will use
[Multi30k dataset from torchtext library](https://pytorch.org/text/stable/datasets.html#multi30k)_
that yields a pair of source-target raw sentences. 

To access torchtext datasets, please install torchdata following instructions at https://github.com/pytorch/data. 




In [2]:
from torchtext.data import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import multi30k, Multi30k
from typing import Iterable, List
import torch.nn.functional as F


multi30k.URL["train"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/training.tar.gz"
multi30k.URL["valid"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/validation.tar.gz"

SRC_LANGUAGE = 'de'
TGT_LANGUAGE = 'en'

# Place-holders
token_transform = {}
vocab_transform = {}


# Create source and target language tokenizer.
token_transform[SRC_LANGUAGE] = get_tokenizer('spacy', language='de_core_news_sm')
token_transform[TGT_LANGUAGE] = get_tokenizer('spacy', language='en_core_web_sm')


# helper function to yield list of tokens
def yield_tokens(data_iter: Iterable, language: str) -> List[str]:
    language_index = {SRC_LANGUAGE: 0, TGT_LANGUAGE: 1}

    for data_sample in data_iter:
        yield token_transform[language](data_sample[language_index[language]])

# Define special symbols and indices
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
# Make sure the tokens are in order of their indices to properly insert them in vocab
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']
 
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    # Training data Iterator 
    train_iter = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    # Create torchtext's Vocab object 
    vocab_transform[ln] = build_vocab_from_iterator(yield_tokens(train_iter, ln),
                                                    min_freq=1,
                                                    specials=special_symbols,
                                                    special_first=True)

# Set UNK_IDX as the default index. This index is returned when the token is not found. 
# If not set, it throws RuntimeError when the queried token is not found in the Vocabulary. 
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
  vocab_transform[ln].set_default_index(UNK_IDX)

## Seq2Seq Network using Transformer

Transformer is a Seq2Seq model introduced in [“Attention is all you
need”](https://papers.nips.cc/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf)_
paper for solving machine translation tasks. 
Below, we will create a Seq2Seq network that uses Transformer. The network
consists of three parts. First part is the embedding layer. This layer converts tensor of input indices
into corresponding tensor of input embeddings. These embedding are further augmented with positional
encodings to provide position information of input tokens to the model. The second part is the 
actual [Transformer](https://pytorch.org/docs/stable/generated/torch.nn.Transformer.html)_ model. 
Finally, the output of Transformer model is passed through linear layer
that give un-normalized probabilities for each token in the target language. 




In [3]:
from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
import math
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# helper Module that adds positional encoding to the token embedding to introduce a notion of word order.
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# helper Module to convert tensor of input indices into corresponding tensor of token embeddings
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# Seq2Seq Network 
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None, 
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

During training, we need a subsequent word mask that will prevent model to look into
the future words when making predictions. We will also need masks to hide
source and target padding tokens. Below, let's define a function that will take care of both. 




In [4]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

Let's now define the parameters of our model and instantiate the same. Below, we also 
define our loss function which is the cross-entropy loss and the optmizer used for training.




In [5]:
torch.manual_seed(0)

SRC_VOCAB_SIZE = len(vocab_transform[SRC_LANGUAGE])
TGT_VOCAB_SIZE = len(vocab_transform[TGT_LANGUAGE])
EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
BATCH_SIZE = 128
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3

transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE, 
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(DEVICE)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

## Collation

As seen in the ``Data Sourcing and Processing`` section, our data iterator yields a pair of raw strings. 
We need to convert these string pairs into the batched tensors that can be processed by our ``Seq2Seq`` network 
defined previously. Below we define our collate function that convert batch of raw strings into batch tensors that
can be fed directly into our model.   




In [6]:
from torch.nn.utils.rnn import pad_sequence

# helper function to club together sequential operations
def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func

# function to add BOS/EOS and create tensor for input sequence indices
def tensor_transform(token_ids: List[int]):
    return torch.cat((torch.tensor([BOS_IDX]), 
                      torch.tensor(token_ids), 
                      torch.tensor([EOS_IDX])))

# src and tgt language text transforms to convert raw strings into tensors indices
text_transform = {}
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    text_transform[ln] = sequential_transforms(token_transform[ln], #Tokenization
                                               vocab_transform[ln], #Numericalization
                                               tensor_transform) # Add BOS/EOS and create tensor


# function to collate data samples into batch tesors
def collate_fn(batch):
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        src_batch.append(text_transform[SRC_LANGUAGE](src_sample.rstrip("\n")))
        tgt_batch.append(text_transform[TGT_LANGUAGE](tgt_sample.rstrip("\n")))

    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    return src_batch, tgt_batch

Let's define training and evaluation loop that will be called for each 
epoch.




In [7]:
from torch.utils.data import DataLoader

def train_epoch(model, optimizer):
    model.train()
    losses = 0
    train_iter = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    train_dataloader = DataLoader(train_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)
    
    for src, tgt in train_dataloader:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        optimizer.zero_grad()

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()

        optimizer.step()
        losses += loss.item()

    return losses / len(list(train_dataloader))


def evaluate(model):
    model.eval()
    losses = 0

    val_iter = Multi30k(split='valid', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    val_dataloader = DataLoader(val_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)

    for src, tgt in val_dataloader:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)
        
        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        losses += loss.item()

    return losses / len(list(val_dataloader))

Now we have all the ingredients to train our model. Let's do it!




In [8]:
from timeit import default_timer as timer
NUM_EPOCHS = 18

for epoch in range(1, NUM_EPOCHS+1):
    start_time = timer()
    train_loss = train_epoch(transformer, optimizer)
    end_time = timer()
    val_loss = evaluate(transformer)
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))


Epoch: 1, Train loss: 5.344, Val loss: 4.114, Epoch time = 45.280s
Epoch: 2, Train loss: 3.761, Val loss: 3.320, Epoch time = 42.511s
Epoch: 3, Train loss: 3.161, Val loss: 2.894, Epoch time = 43.232s
Epoch: 4, Train loss: 2.768, Val loss: 2.638, Epoch time = 44.267s
Epoch: 5, Train loss: 2.480, Val loss: 2.441, Epoch time = 42.927s
Epoch: 6, Train loss: 2.250, Val loss: 2.315, Epoch time = 42.951s
Epoch: 7, Train loss: 2.060, Val loss: 2.201, Epoch time = 44.267s
Epoch: 8, Train loss: 1.897, Val loss: 2.113, Epoch time = 43.378s
Epoch: 9, Train loss: 1.754, Val loss: 2.058, Epoch time = 43.213s
Epoch: 10, Train loss: 1.631, Val loss: 2.002, Epoch time = 42.655s
Epoch: 11, Train loss: 1.524, Val loss: 1.975, Epoch time = 43.084s
Epoch: 12, Train loss: 1.420, Val loss: 1.945, Epoch time = 42.723s
Epoch: 13, Train loss: 1.333, Val loss: 1.967, Epoch time = 43.049s
Epoch: 14, Train loss: 1.251, Val loss: 1.942, Epoch time = 43.018s
Epoch: 15, Train loss: 1.173, Val loss: 1.928, Epoch time

## (5 points) Decoding functions
---

### Greedy decoding

In [9]:
# function to generate output sequence using greedy algorithm 
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    """
    Greedy decoding algorithm

    Args:
        model: model
        src: source sentence
        src_mask: source mask
        max_len: maximum length of output sentence
        start_symbol: start symbol

    Returns:
        output sentence
    """

    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len-1):
        memory = memory.to(DEVICE)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                    .type(torch.bool)).to(DEVICE)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
            break
    return ys

### Top K sampling with temperature

In [10]:
import torch.nn.functional as F

def top_k_sampling_temperature(model, src, src_mask, max_len, start_symbol, k=5, temperature=1.0):
    """
    Top-k sampling with temperature

    Args:
        model: model
        src: source sentence
        src_mask: source mask
        max_len: maximum length of output sentence
        start_symbol: start symbol
        k: top-k sampling parameter
        temperature: float value between 0 and 1

    Returns:
        output sentence
    """
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len-1):
        memory = memory.to(DEVICE)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                    .type(torch.bool)).to(DEVICE)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        prob = F.softmax(prob / temperature, dim=1)
        top_k_prob, top_k_index = torch.topk(prob, k=k, dim=1)
        random_index = torch.multinomial(top_k_prob, 1)
        
        next_word = top_k_index[0][random_index].item()
        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
            break
    return ys

### Top k sampling

On réutilise le *top k sampling* avec temperature en définissant la température à 1.0

In [11]:
def top_k_sampling(model, src, src_mask, max_len, start_symbol, k=5):
    """
    Top-k sampling without temperature

    Args:
        model: model
        src: source sentence
        src_mask: source mask
        max_len: maximum length of output sentence
        start_symbol: start symbol
        k: top-k sampling parameter

    Returns:
        output sentence
    """
    return top_k_sampling_temperature(model, src, src_mask, max_len, start_symbol, k=k, temperature=1.0)


### Beam Search

In [12]:
def beam_search_sampling(model, src, src_mask, max_len, start_symbol, k=5):
    """
    Beam search sampling

    Args:
        model: model
        src: source sentence
        src_mask: source mask
        max_len: maximum length of output sentence
        start_symbol: start symbol
        k: beam size

    Returns:
        output sentence
    """
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)

    memory = model.encode(src, src_mask)

    # initialize the first hypothesis
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    memory = memory.to(DEVICE)
    tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                .type(torch.bool)).to(DEVICE)
    out = model.decode(ys, memory, tgt_mask)
    out = out.transpose(0, 1)
    prob = model.generator(out[:, -1])
    prob = F.softmax(prob, dim=1)
    top_k_prob, top_k_index = torch.topk(prob, k=k, dim=1)
 
    # initialize the rest of the hypothesis
    hypothesis = []
    for i in range(k):
        next_word = top_k_index[0][i].item()
        hypothesis.append({
            "sequence": torch.cat([ys,
                    torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0),
            "score": top_k_prob[0][i].item(),
            "memory": memory
        })

    for i in range(max_len-1):
        new_hypothesis = []
        for h in hypothesis:
            ys = h["sequence"]
            memory = h["memory"]
            tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                        .type(torch.bool)).to(DEVICE)
            out = model.decode(ys, memory, tgt_mask)
            out = out.transpose(0, 1)
            prob = model.generator(out[:, -1])
            prob = F.softmax(prob, dim=1)
            top_k_prob, top_k_index = torch.topk(prob, k=k, dim=1)

            for j in range(k):
                next_word = top_k_index[0][j].item()
                new_hypothesis.append({
                    "sequence": torch.cat([ys,
                                           torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0),
                    "score": h["score"] * top_k_prob[0][j].item(),
                    "memory": memory
                })
        new_hypothesis = sorted(new_hypothesis, key=lambda x: x["score"], reverse=True)
        hypothesis = new_hypothesis[:k]
        if hypothesis[0]["sequence"][-1] == EOS_IDX:
            break
    return hypothesis[0]["sequence"]

## Traduction

In [13]:
from typing import Callable

# actual function to translate input sentence into target language
def translate(model: torch.nn.Module, decoding_function: Callable, src_sentence: str, **kwargs):
    """
    Translate sentence using any decoding strategy,

    Args:
        model: model,
        decoding_function: callable function to generate sentences
        src_sentence: source sentence
        **kwargs: arguments of the decoding function

    Returns:
        translated sentence
    """
    model.eval()
    src = text_transform[SRC_LANGUAGE](src_sentence).view(-1, 1)
    num_tokens = src.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = decoding_function(
        model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX, **kwargs).flatten()
    return " ".join(vocab_transform[TGT_LANGUAGE].lookup_tokens(list(tgt_tokens.cpu().numpy())))\
              .replace("<bos>", "")\
              .replace("<eos>", "")

Maintenant comparons la traduction pour chacune des méthodes définies ci-dessus

In [14]:
def multiple_translations(text):
    """
    Generate multiple translations for a given sentence

    Args:
        text: sentence to translate
    """

    print("Greedy :", translate(transformer, greedy_decode, text))
    print("Top K=10 :", translate(transformer, top_k_sampling, text, k=10))
    print("Top K=10 Temp=0.4 :", translate(transformer, top_k_sampling_temperature, text, k=10, temperature=0.4))
    print("Beam Search k=5 :", translate(transformer, beam_search_sampling, text, k=5))

### Exemple : "*A group of people standing in front of an igloo .*"


In [15]:
multiple_translations("Eine Gruppe von Menschen steht vor einem Iglu .")

Greedy :  A group of people standing in front of an igloo . 
Top K=10 :  A group of people standing outside an igloo . 
Top K=10 Temp=0.4 :  A group of people standing in front of an igloo . 
Beam Search k=5 :  A group of people standing in front of an igloo . 


### Exemple : "*This translation will never be as accurate as mine.*"

In [16]:
multiple_translations("Diese Übersetzung wird niemals so genau sein wie meine .")

Greedy :  This female athlete is being kissed by his talent . 
Top K=10 :  This female runner is skiing and gesturing . 
Top K=10 Temp=0.4 :  This female runner is skiing , while being washed . 
Beam Search k=5 :  This female athlete is skiing , wearing matching shoes . 


On observe que le modèle rencontre des difficultés à traduire des phrases hors données d'entrainements. Cela s'explique par le fait qu'il a été entrainé sur seulement 30k exemples ce qui est peu pour obtenir un vocabulaire correspondant à un language complet.

## (2 points) Compute the BLEU score of the model
---

Use the sacreBLEU implementation to evaluate your model and quantitatively compare the 4 implemented decoding approaches. Explain what all the output values mean (when using the `corpus_score` function).

In [17]:
from sacrebleu.metrics import BLEU
bleu = BLEU()

In [18]:
test_data = Multi30k(split='valid', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))

# sources is the list of sentences of the source language
# refs is the list of the same sentences in the target language
sources, refs = zip(*test_data)
refs = [refs]

trans_greedy = [translate(transformer, greedy_decode, src) for src in sources]
trans_topk = [translate(transformer, top_k_sampling, src, k=10) for src in sources]
trans_topk_temp = [translate(transformer, top_k_sampling_temperature, src, k=10, temperature=0.4) for src in sources]
trans_beam = [translate(transformer, beam_search_sampling, src, k=5) for src in sources]

In [19]:
print("Greedy translation BLEU score :")
bleu.corpus_score(trans_greedy, refs)

Greedy translation BLEU score :


BLEU = 35.30 67.2/43.2/28.2/18.9 (BP = 1.000 ratio = 1.009 hyp_len = 13405 ref_len = 13289)

- Le premier nombre correspond au score BLEU
- les quatre nombres X/X/X/X representent la précision pour 1–4 ngram
- BP (brevity penalty) représente le score qui pénalise les traductions trop petites
- `ratio` indique le ratio entre la taille de l'hypothèse et la taille de la ref
- `hyp_len` se réfère au nombre total de caractères pour le texte de l'hypothèse
- `ref_len` est le nombre total de caractères du texte de référence

In [20]:
print("Top K sampling translation BLEU score :")
print(bleu.corpus_score(trans_topk, refs))

print("Top K sampling with temperature translation BLEU score :")
print(bleu.corpus_score(trans_topk_temp, refs))

print("Beam Search translation BLEU score :")
print(bleu.corpus_score(trans_beam, refs))

Top K sampling translation BLEU score :
BLEU = 28.97 61.9/36.6/22.4/13.9 (BP = 1.000 ratio = 1.015 hyp_len = 13489 ref_len = 13289)
Top K sampling with temperature translation BLEU score :
BLEU = 34.51 66.7/42.4/27.6/18.2 (BP = 1.000 ratio = 1.008 hyp_len = 13390 ref_len = 13289)
Beam Search translation BLEU score :
BLEU = 36.82 69.7/45.8/30.4/20.7 (BP = 0.978 ratio = 0.978 hyp_len = 12998 ref_len = 13289)


Beam Search semble être sensiblement meilleur que les autres techniques de décodages.